В формулировке заданий будет использоваться понятие **worker**. Это слово обозначает какую-то единицу параллельного выполнения, в случае питона это может быть **поток** или **процесс**, выбирайте то, что лучше будет подходить к конкретной задаче

# Задание 1 (7 баллов)

В одном из заданий по ML от вас требовалось написать кастомную реализацию Random Forest. Её проблема состоит в том, что она работает медленно, так как использует всего один поток для работы. Добавление параллельного программирования в код позволит получить существенный прирост в скорости обучения и предсказаний.

В данном задании от вас требуется добавить возможность обучать случайный лес параллельно и использовать параллелизм для предсказаний. Для этого вам понадобится:
1. Добавить аргумент `n_jobs` в метод `fit`. `n_jobs` показывает количество worker'ов, используемых для распараллеливания
2. Добавить аргумент `n_jobs` в методы `predict` и `predict_proba`
3. Реализовать функционал по распараллеливанию в данных методах

В результате код `random_forest.fit(X, y, n_jobs=2)` и `random_forest.predict(X, y, n_jobs=2)` должен работать в ~1.5-2 раза быстрее, чем `random_forest.fit(X, y, n_jobs=1)` и `random_forest.predict(X, y, n_jobs=1)` соответственно

Если у вас по каким-то причинам нет кода случайного леса из ДЗ по ML, то вы можете написать его заново или попросить у однокурсника. *Детали* реализации ML части оцениваться не будут, НО, если вы поломаете логику работы алгоритма во время реализации параллелизма, то за это будут сниматься баллы

В задании можно использовать только модули из **стандартной библиотеки** питона, а также функции и классы из **sklearn** при помощи которых вы изначально писали лес

In [13]:
import random
from concurrent.futures import ThreadPoolExecutor
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification


class RandomForestClassifierCustom(BaseEstimator):
    def __init__(
        self, n_estimators=10, max_depth=None, max_features=None, random_state=None
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state

        self.trees = []
        self.feat_ids_by_tree = []
        
        
    @staticmethod    
    def check_jobs(n_jobs, n_estimators):
        if n_jobs:
            return min(n_jobs, n_estimators)
        elif not n_jobs:
            return 1
        
        
    @staticmethod
    def calc_estim_per_job(n_estimators, n_jobs):
        n_estimators_per_job = np.full(n_jobs, n_estimators // n_jobs, dtype=int)
        n_estimators_per_job[: n_estimators % n_jobs] += 1
        
        # some calculations for building the same trees regardless n_jobs
        lst_cumsum = np.insert(np.cumsum(n_estimators_per_job), 0, 0, axis=0)
        # list for start and stop for calculating seeds
        seed_coords = []
        for i in range(1, len(lst_cumsum)):
            seed_coords.append((lst_cumsum[i-1], lst_cumsum[i]))
        return n_estimators_per_job, seed_coords
            
    
    def build_trees(self, seed_coords, X, y):
        start, stop = seed_coords
        trees = []
        feat_ids_by_tree = []
        for i in range(start, stop):
            # setting seed
            np.random.seed(self.random_state + i)

            # selecting n random features 
            feat_ids = np.random.choice(X.shape[1], size=self.max_features, replace=False)
            feat_ids_by_tree.append(feat_ids)

            # creating pseudosample using bootstrap
            # random indices for pseudosample
            pseudo_idx = np.random.choice(X.shape[0], size=X.shape[0], replace=True).reshape(X.shape[0], 1)
            pseudo_X = X[pseudo_idx, feat_ids]
            pseudo_y = y[pseudo_idx]

            # creating and fitting model
            dec_tree_class = DecisionTreeClassifier(max_depth=self.max_depth,
                                                    max_features=self.max_features, 
                                                    random_state=self.random_state)
            dec_tree_class.fit(pseudo_X, pseudo_y)
            trees.append(dec_tree_class)
        return feat_ids_by_tree, trees

    
    def __do_parallel(self, func, n_jobs, *args): # args may contain X or both X and y
        n_jobs = RandomForestClassifierCustom.check_jobs(n_jobs, self.n_estimators)
        n_est_per_job, seed_coords = RandomForestClassifierCustom.calc_estim_per_job(self.n_estimators, n_jobs)
        futures = []
        with ThreadPoolExecutor() as pool:
            for i, _ in enumerate(n_est_per_job):
                futures.append(pool.submit(func, seed_coords[i], *args))
        return futures


    def fit(self, X, y, n_jobs=None):
        self.classes_ = sorted(np.unique(y))
        futures = self.__do_parallel(self.build_trees, n_jobs, X, y)
        for future in futures:
            self.feat_ids_by_tree.extend(future.result()[0]) 
            self.trees.extend(future.result()[1])
        return self
            
        
    def parallel_predict_proba(self, seed_coords, X):
        start, stop = seed_coords
        probs = []
        for i in range(start, stop):
            proba = self.trees[i].predict_proba(X[:, self.feat_ids_by_tree[i]])
            probs.append(proba)
        return probs
        
    
    def predict_proba(self, X, n_jobs=None):
        futures = self.__do_parallel(self.parallel_predict_proba, n_jobs, X)
        result = []
        for future in futures:
            result.extend(future.result())
        return np.mean(result, axis=0)
    
    
    def predict(self, X, n_jobs=None):
        probas = self.predict_proba(X, n_jobs)
        predictions = np.argmax(probas, axis=1)
        return predictions
    
    
X, y = make_classification(n_samples=100000)

In [27]:
random_forest1 = RandomForestClassifierCustom(max_depth=30, n_estimators=10, max_features=2, random_state=42)

In [28]:
%%time

_ = random_forest1.fit(X, y, n_jobs=1)

CPU times: user 5.03 s, sys: 0 ns, total: 5.03 s
Wall time: 5.04 s


In [29]:
%%time

preds_1 = random_forest1.predict(X, n_jobs=1)

CPU times: user 135 ms, sys: 0 ns, total: 135 ms
Wall time: 143 ms


In [30]:
random_forest1.feat_ids_by_tree

[array([ 0, 17]),
 array([9, 6]),
 array([15, 14]),
 array([12, 16]),
 array([1, 3]),
 array([10, 14]),
 array([1, 9]),
 array([14, 19]),
 array([15,  8]),
 array([ 1, 12])]

In [31]:
random_forest2 = RandomForestClassifierCustom(max_depth=30, n_estimators=10, max_features=2, random_state=42)

In [32]:
%%time

_ = random_forest2.fit(X, y, n_jobs=2)

CPU times: user 5.09 s, sys: 0 ns, total: 5.09 s
Wall time: 2.69 s


In [33]:
%%time

preds_2 = random_forest2.predict(X, n_jobs=2)

CPU times: user 136 ms, sys: 0 ns, total: 136 ms
Wall time: 78.8 ms


In [34]:
random_forest2.feat_ids_by_tree

[array([ 0, 17]),
 array([9, 6]),
 array([15, 14]),
 array([12, 16]),
 array([1, 3]),
 array([10, 14]),
 array([1, 9]),
 array([14, 19]),
 array([15,  8]),
 array([ 1, 12])]

In [35]:
(preds_1 == preds_2).all()   # Количество worker'ов не должно влиять на предсказания

True

#### Какие есть недостатки у вашей реализации параллельного Random Forest (если они есть)? Как это можно исправить? Опишите словами, можно без кода (+1 дополнительный балл)

Често говоря, я путаюсь в типах методов, скорее всего, я неправильно применила `@staticmethod`, но пусть будет как есть, можно ругаться:) В целом, самый большой недостаток в том, что я возвращаю futures как результат распараллеливания функций, не знаю, можно ли так, конечно, можно доработать, но эти деревья выпили из меня жизнь. Я постаралась, чтобы зафиксировать одинаковый seed для каждого дерева вне зависимости от количества workers, а сами деревья сделать разными со своим фиксированным seed. В ячейках выше видно, что деревья создавались по одинаковому набору фичей вне зависимости от количества worker (вывела аттрибут `feat_ids_by_tree`).

# Задание 2 (9 баллов)

Напишите декоратор `memory_limit`, который позволит ограничивать использование памяти декорируемой функцией.

Декоратор должен принимать следующие аргументы:
1. `soft_limit` - "мягкий" лимит использования памяти. При превышении функцией этого лимита должен будет отображён **warning**
2. `hard_limit` - "жёсткий" лимит использования памяти. При превышении функцией этого лимита должно будет брошено исключение, а функция должна немедленно завершить свою работу
3. `poll_interval` - интервал времени (в секундах) между проверками использования памяти

Требования:
1. Потребление функцией памяти должно отслеживаться **во время выполнения функции**, а не после её завершения
2. **warning** при превышении `soft_limit` должен отображаться один раз, даже если функция переходила через этот лимит несколько раз
3. Если задать `soft_limit` или `hard_limit` как `None`, то соответствующий лимит должен быть отключён
4. Лимиты должны передаваться и отображаться в формате `<number>X`, где `X` - символ, обозначающий порядок единицы измерения памяти ("B", "K", "M", "G", "T", ...)
5. В тексте warning'ов и исключений должен быть указан текщий объём используемой памяти и величина превышенного лимита

В задании можно использовать только модули из **стандартной библиотеки** питона, можно писать вспомогательные функции и/или классы

В коде ниже для вас предопределены некоторые полезные функции, вы можете ими пользоваться, а можете не пользоваться

In [18]:
import os
import sys
import psutil
import time
import warnings
import threading


def get_memory_usage():    # Показывает текущее потребление памяти процессом
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss


def human_readble_to_bytes(limit):
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for idx, s in enumerate(symbols):
        prefix[s] = 1 << (idx + 1) * 10
    for s in symbols:
        if limit[-1] == s:
            value = int(float(limit[:-1])*prefix[s])
            return value
    return int(limit)


def bytes_to_human_readable(n_bytes):
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for idx, s in enumerate(symbols):
        prefix[s] = 1 << (idx + 1) * 10
    for s in reversed(symbols):
        if n_bytes >= prefix[s]:
            value = float(n_bytes) / prefix[s]
            return f"{value:.2f}{s}"
    return f"{n_bytes}B"


def memory_limit(softcap=None, hardcap=None, poll_interval=1):
    # Ваш код здесь
    pass

In [36]:
import os
import sys
import psutil
import time
import warnings
from threading import Thread
import _thread

class MemoryThread(Thread):
    
    symbols = ('K', 'M', 'G', 'T', 'P', 'E', 'Z', 'Y')
    prefix = {}
    for idx, s in enumerate(symbols):
        prefix[s] = 1 << (idx + 1) * 10
    
    
    def __init__(self, soft_limit, hard_limit, poll_interval):
        super().__init__()
        self.soft_limit = soft_limit
        self.hard_limit = hard_limit
        self.poll_interval = poll_interval
        self.broken_soft = False
        if soft_limit:
            self._bytes_soft_limit = self.human_readble_to_bytes(self.soft_limit)
        if hard_limit:
            self._bytes_hard_limit = self.human_readble_to_bytes(self.hard_limit)
                   
        
    def get_memory_usage(self):    # Показывает текущее потребление памяти процессом
        process = psutil.Process(os.getpid())
        mem_info = process.memory_info()
        return mem_info.rss


    def human_readble_to_bytes(self, limit):
        for s in MemoryThread.symbols:
            if limit[-1] == s:
                value = int(float(limit[:-1]) * MemoryThread.prefix[s])
                return value
        return int(limit)


    def bytes_to_human_readable(self, n_bytes):
        for s in reversed(MemoryThread.symbols):
            if n_bytes >= MemoryThread.prefix[s]:
                value = float(n_bytes) / MemoryThread.prefix[s]
                return f"{value:.2f}{s}"
        return f"{n_bytes}B"

        
    def run(self):
        while True:
            memory_usage = self.get_memory_usage()
            self.memory_usage = self.bytes_to_human_readable(memory_usage)
            if (not self.broken_soft) and self.soft_limit:
                self.check_soft(memory_usage)
            if self.hard_limit:
                self.check_hard(memory_usage)
            if hasattr(self, 'event'):
                return
            time.sleep(self.poll_interval)
        return
            
                      
    def check_soft(self, memory_usage):
        def custom_formatwarning(msg, *args, **kwargs):
            # ignore everything except the message
            return  f"Warning! {str(msg)}"
        
        if memory_usage >= self._bytes_soft_limit:
            self.broken_soft = True
            warnings.formatwarning = custom_formatwarning
            warnings.warn(
                f"Soft limit {self.soft_limit} is broken, current memory usage - {self.memory_usage}!", 
                UserWarning
            )
            
            
    def check_hard(self, memory_usage):
        if memory_usage >= self._bytes_hard_limit:
            _thread.interrupt_main() # I tried to catch KeybordInterrupt error here too
            time.sleep(2)
            # I used it here to restart kernel for cleaning memory
            # for some reason, gc.collect() could not cope with this task 
            os._exit(0)
            
            
class MemLimitException(BaseException):
    def __init__(self, obj):
        self.obj = obj
    
    
    def __str__(self):
        return f"Hard limit {self.obj.hard_limit} is broken, current memory usage - {self.obj.memory_usage}!"

            
def memory_limit(soft_limit="512M", hard_limit="1.5G", poll_interval=0.1):
    def decor(func):
        def inner_func(*args, **kwargs):
            thread_started = False
            if soft_limit or hard_limit:
                thread_started = True
                mem_check = MemoryThread(soft_limit, hard_limit, poll_interval)
                mem_check.start()
            try:
                result = func(*args, **kwargs)
                if thread_started:
                    mem_check.event = True
            except KeyboardInterrupt as e: # error is not catched
                raise MemLimitException(mem_check)
            return result
        return inner_func
    return decor


@memory_limit(soft_limit="512M", hard_limit="1.5G", poll_interval=0.1)
def memory_increment():
    """
    Функция для тестирования
    
    В течение нескольких секунд достигает использования памяти 1.89G
    Потребление памяти и скорость накопления можно варьировать, изменяя код
    """
    lst = []
    for i in range(50000000):
        if i % 500000 == 0:
            time.sleep(0.1)
        lst.append(i)
    return lst

memory_increment()

Warning! Soft limit 512M is broken, current memory usage - 524.98M!

MemLimitException: Hard limit 1.5G is broken, current memory usage - 1.51G!

# Задание 3 (11 баллов)

Напишите функцию `parallel_map`. Это должна быть **универсальная** функция для распараллеливания, которая эффективно работает в любых условиях.

Функция должна принимать следующие аргументы:
1. `target_func` - целевая функция (обязательный аргумент)
2. `args_container` - контейнер с позиционными аргументами для `target_func` (по-умолчанию `None` - позиционные аргументы не передаются)
3. `kwargs_container` - контейнер с именованными аргументами для `target_func` (по-умолчанию `None` - именованные аргументы не передаются)
4. `n_jobs` - количество workers, которые будут использованы для выполнения (по-умолчанию `None` - количество логических ядер CPU в системе)

Функция должна работать аналогично `***PoolExecutor.map`, применяя функцию к переданному набору аргументов, но с некоторыми дополнениями и улучшениями
    
Поскольку мы пишем **универсальную** функцию, то нам нужно будет выполнить ряд требований, чтобы она могла логично и эффективно работать в большинстве ситуаций

1. `target_func` может принимать аргументы любого вида в любом количестве
2. Любые типы данных в `args_container`, кроме `tuple`, передаются в `target_func` как единственный позиционный аргумент. `tuple` распаковываются в несколько аргументов
3. Количество элементов в `args_container` должно совпадать с количеством элементов в `kwargs_container` и наоборот, также значение одного из них или обоих может быть равно `None`, в иных случаях должна кидаться ошибка (оба аргумента переданы, но размеры не совпадают)

4. Функция должна выполнять определённое количество параллельных вызовов `target_func`, это количество зависит от числа переданных аргументов и значения `n_jobs`. Сценарии могут быть следующие
    + `args_container=None`, `kwargs_container=None`, `n_jobs=None`. В таком случае функция `target_func` выполнится параллельно столько раз, сколько на вашем устройстве логических ядер CPU
    + `args_container=None`, `kwargs_container=None`, `n_jobs=5`. В таком случае функция `target_func` выполнится параллельно **5** раз
    + `args_container=[1, 2, 3]`, `kwargs_container=None`, `n_jobs=5`. В таком случае функция `target_func` выполнится параллельно **3** раза, несмотря на то, что `n_jobs=5` (так как есть всего 3 набора аргументов для которых нам нужно получить результат, а лишние worker'ы создавать не имеет смысла)
    + `args_container=None`, `kwargs_container=[{"s": 1}, {"s": 2}, {"s": 3}]`, `n_jobs=5`. Данный случай аналогичен предыдущему, но здесь мы используем именованные аргументы
    + `args_container=[1, 2, 3]`, `kwargs_container=[{"s": 1}, {"s": 2}, {"s": 3}]`, `n_jobs=5`. Данный случай аналогичен предыдущему, но здесь мы используем и позиционные, и именованные аргументы
    + `args_container=[1, 2, 3, 4]`, `kwargs_container=None`, `n_jobs=2`. В таком случае в каждый момент времени параллельно будет выполняться **не более 2** функций `target_func`, так как нам нужно выполнить её 4 раза, но у нас есть только 2 worker'а.
    + В подобных случаях (из примера выше) должно оптимизироваться время выполнения. Если эти 4 вызова выполняются за 5, 1, 2 и 1 секунды, то параллельное выполнение с `n_jobs=2` должно занять **5 секунд** (не 7 и тем более не 10)

5. `parallel_map` возвращает результаты выполнения `target_func` **в том же порядке**, в котором были переданы соответствующие аргументы
6. Работает с функциями, созданными внутри других функций

Для базового решения от вас не ожидается **сверххорошая** оптимизация по времени и памяти для всех возможных случаев. Однако за хорошо оптимизированную логику работы можно получить до **+3 дополнительных баллов**

Вы можете сделать класс вместо функции, если вам удобнее

В задании можно использовать только модули из **стандартной библиотеки** питона

Ниже приведены тестовые примеры по каждому из требований

In [72]:
import multiprocessing


# def parallel_map(target_func,
#                  args_container=None,
#                  kwargs_container=None,
#                  n_jobs=None):
    # Ваш код здесь
    
class ParallelMap:
    def __init__(self, target_func, args_container=None,
                 kwargs_container=None, n_jobs=None):
        self.target_func = target_func
        self.args_container = args_container
        self.kwargs_container = kwargs_container
        self.n_jobs = n_jobs
        self.__return_dict = None
        
    
    def set_manager(self):
        manager = multiprocessing.Manager()
        return_dict = manager.dict()
        return self.__return_dict
    
    
    def __check_jobs(self):
        n_args = len(self.args_container)
        if self.n_jobs:
            return min(n_jobs, n_args)
        elif not n_jobs:
            return multiprocessing.cpu_count()
        
        
    def __check_len_match(self):
        if self.args_container and self.kwargs_container:
            if len(self.args_container) != len(self.kwargs_container):
                raise(TypeError,'Unmatched length of args and kwargs!')
            self.target_func(*self.args_container, **self.kwargs_container)
        elif self.args_container:
            self.target_func(*self.args_container)
        elif self.kwargs_container:
            self.target_func(**self.kwargs_container)
        else:
            self.target_func()
            
            
    def a(func, i, manager_dict):
        def b(*args, **kwargs):
            print('111111111')
            result = func(*args, **kwargs)
            print(result)
            manager_dict[i] = result
            print(manager_dict)
            print('222222222')
        return b

In [45]:
def worker(procnum, return_dict):
    """worker function"""
    print(str(procnum) + " represent!")
    return_dict[procnum] = procnum


manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = []
for i in range(5):
    p = multiprocessing.Process(target=worker, args=(i, return_dict))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()
print(return_dict.values())

0 represent!
1 represent!
2 represent!
3 represent!
4 represent!
[0, 1, 2, 3, 4]


In [56]:
manager = multiprocessing.Manager()
manager_dict = manager.dict()
def a(func, i, manager_dict):
    def b(*args, **kwargs):
        print('111111111')
        result = func(*args, **kwargs)
        print(result)
        manager_dict[i] = result
        print(manager_dict)
        print('222222222')
    return b

args_container=2
decord_func = a(test_func, 0, manager_dict)

decord_func(args_container)
manager_dict.values()

111111111
7
{0: 7}
222222222


[7]

In [101]:
# Пример 6
# Работает с функциями, созданными внутри других функций
def test_func3():
    def inner_test_func(sleep_time):
        time.sleep(sleep_time)
        return 1
    return main(inner_test_func, 3)
test_func3()

<function test_func3.<locals>.inner_test_func at 0x7fe4c55bdb80>
oooooooo


NameError: name 'b' is not defined

In [3]:
import numpy as np


n_jobs, n_args = 2, 4
n_funcs_per_job = np.full(n_jobs, n_args // n_jobs, dtype=int)
n_funcs_per_job[: n_args % n_jobs] += 1

# some calculations for building the same trees regardless n_jobs
lst_cumsum = np.insert(np.cumsum(n_funcs_per_job), 0, 0, axis=0)
# list for start and stop for calculating seeds
args_coords = []
for i in range(1, len(lst_cumsum)):
    args_coords.append((lst_cumsum[i-1], lst_cumsum[i]))
    
n_funcs_per_job, args_coords

(array([2, 2]), [(0, 2), (2, 4)])

In [47]:
import time


# Это только один пример тестовой функции, ваша parallel_map должна уметь эффективно работать с ЛЮБЫМИ функциями
# Поэтому обязательно протестируйте код на чём-нибудбь ещё
def test_func(x=1, s=2, a=1, b=1, c=1):
    time.sleep(s)
    return a*x**2 + b*x + c

In [ ]:
%%time

# Пример 2.1
# Отдельные значения в args_container передаются в качестве позиционных аргументов
parallel_map(test_func, args_container=[1, 2.0, 3j-1, 4])   # Здесь происходят параллельные вызовы: test_func(1) test_func(2.0) test_func(3j-1) test_func(4)

CPU times: user 395 µs, sys: 8.26 ms, total: 8.65 ms
Wall time: 2.01 s


[3, 7.0, (-8-3j), 21]

In [ ]:
%%time

# Пример 2.2
# Элементы типа tuple в args_container распаковываются в качестве позиционных аргументов
parallel_map(test_func, [(1, 1), (2.0, 2), (3j-1, 3), 4])    # Здесь происходят параллельные вызовы: test_func(1, 1) test_func(2.0, 2) test_func(3j-1, 3) test_func(4)

CPU times: user 7.18 ms, sys: 7.73 ms, total: 14.9 ms
Wall time: 3.01 s


[3, 7.0, (-8-3j), 21]

In [ ]:
%%time

# Пример 3.1
# Возможна одновременная передача args_container и kwargs_container, но количества элементов в них должны быть равны
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}, {"s": 3}])

# Здесь происходят параллельные вызовы: test_func(1, s=3) test_func(2, s=3) test_func(3, s=3) test_func(4, s=3)

CPU times: user 5.89 ms, sys: 8.84 ms, total: 14.7 ms
Wall time: 3.02 s


[3, 7, 13, 21]

In [ ]:
%%time

# Пример 3.2
# args_container может быть None, а kwargs_container задан явно
parallel_map(test_func,
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}, {"s": 3}])

CPU times: user 6.54 ms, sys: 6.06 ms, total: 12.6 ms
Wall time: 3.02 s


[3, 3, 3, 3]

In [ ]:
%%time

# Пример 3.3
# kwargs_container может быть None, а args_container задан явно
parallel_map(test_func,
             args_container=[1, 2, 3, 4])

CPU times: user 4.11 ms, sys: 9.2 ms, total: 13.3 ms
Wall time: 2.01 s


[3, 7, 13, 21]

In [ ]:
%%time

# Пример 3.4
# И kwargs_container, и args_container могут быть не заданы
parallel_map(test_func)

CPU times: user 500 µs, sys: 43.3 ms, total: 43.8 ms
Wall time: 2.04 s


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
%%time

# Пример 3.4
# И kwargs_container, и args_container могут быть не заданы
parallel_map(test_func)

CPU times: user 500 µs, sys: 43.3 ms, total: 43.8 ms
Wall time: 2.04 s


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
%%time

# Пример 3.5
# При несовпадении количеств позиционных и именованных аргументов кидается ошибка
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}])

ValueError: Numbers of positional arguments and keyword arguments do not match: 4 and 3

In [ ]:
%%time

# Пример 4.1
# Если функция не имеет обязательных аргументов и аргумент n_jobs не был передан, то она выполняется параллельно столько раз, сколько ваш CPU имеет логических ядер
# В моём случае это 24, у вас может быть больше или меньше
parallel_map(test_func)

CPU times: user 9.3 ms, sys: 51.2 ms, total: 60.5 ms
Wall time: 2.06 s


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
%%time

# Пример 4.2
# Если функция не имеет обязательных аргументов и передан только аргумент n_jobs, то она выполняется параллельно n_jobs раз
parallel_map(test_func, n_jobs=2)

CPU times: user 2.06 ms, sys: 5.92 ms, total: 7.99 ms
Wall time: 2.01 s


[3, 3]

In [ ]:
%%time

# Пример 4.3
# Если аргументов для target_func указано МЕНЬШЕ, чем n_jobs, то используется такое же количество worker'ов, сколько было передано аргументов
parallel_map(test_func,
             args_container=[1, 2, 3],
             n_jobs=5)   # Здесь используется 3 worker'a

CPU times: user 314 µs, sys: 8.69 ms, total: 9 ms
Wall time: 2.01 s


[3, 7, 13]

In [ ]:
%%time

# Пример 4.4
# Аналогичный предыдущему случай, но с именованными аргументами
parallel_map(test_func,
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}],
             n_jobs=5)   # Здесь используется 3 worker'a

CPU times: user 1.26 ms, sys: 9.47 ms, total: 10.7 ms
Wall time: 3.01 s


[3, 3, 3]

In [ ]:
%%time

# Пример 4.5
# Комбинация примеров 4.3 и 4.4 (переданы и позиционные и именованные аргументы)
parallel_map(test_func,
             args_container=[1, 2, 3],
             kwargs_container=[{"s": 3}, {"s": 3}, {"s": 3}],
             n_jobs=5)   # Здесь используется 3 worker'a

CPU times: user 7.88 ms, sys: 0 ns, total: 7.88 ms
Wall time: 3.01 s


[3, 7, 13]

In [ ]:
%%time

# Пример 4.6
# Если аргументов для target_func указано БОЛЬШЕ, чем n_jobs, то используется n_jobs worker'ов
parallel_map(test_func,
             args_container=[1, 2, 3, 4],
             kwargs_container=None,
             n_jobs=2)   # Здесь используется 2 worker'a

CPU times: user 7.88 ms, sys: 0 ns, total: 7.88 ms
Wall time: 3.01 s


[3, 7, 13]

In [ ]:
%%time

# Пример 4.7
# Время выполнения оптимизируется, данный код должен отрабатывать за 5 секунд
parallel_map(test_func,
             kwargs_container=[{"s": 5}, {"s": 1}, {"s": 2}, {"s": 1}],
             n_jobs=2)

CPU times: user 3.03 ms, sys: 11 ms, total: 14 ms
Wall time: 5.01 s


[3, 3, 3, 3]

In [ ]:
def test_func2(string, sleep_time=1):
    time.sleep(sleep_time)
    return string

# Пример 5
# Результаты возвращаются в том же порядке, в котором были переданы соответствующие аргументы вне зависимости от того, когда завершился worker
arguments = ["first", "second", "third", "fourth", "fifth"]
parallel_map(test_func2,
             args_container=arguments,
             kwargs_container=[{"sleep_time": 5}, {"sleep_time": 4}, {"sleep_time": 3}, {"sleep_time": 2}, {"sleep_time": 1}])

['first', 'second', 'third', 'fourth', 'fifth']

In [ ]:
%%time


def test_func3():
    def inner_test_func(sleep_time):
        time.sleep(sleep_time)
    return parallel_map(inner_test_func, args_container=[1, 2, 3])

# Пример 6
# Работает с функциями, созданными внутри других функций
test_func3()

[None, None, None]